# EDA and Cleaning

In [1]:
#importing important libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [2]:
#load in data set
train = pd.read_csv('./datasets/train.csv')

## Begin EDA

In [3]:
train.shape

(2051, 81)

In [4]:
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [5]:
train.columns = train.columns.str.replace(' ', '_')
#using code from our class ames lesson to get rid of spacing in column names 

In [6]:
train.columns = train.columns.str.lower() #decided to go ahead and lower names as well

## Data Cleaning

### Addressing ordinal missing values

- We know from the data descriptions that null values for basements, garages, masonry, and fireplaces means that there is no basement, garage, masonry, or fireplace. So going to fill those null values with 0's
- Going to be creating functions to to easily change any ordinal categorical data to numerical data based on specific ranking systems for values I believe to play a huge role in pricing that we can use on both training and testing data

In [8]:
train.isnull().sum()[train.isnull().sum() != 0]

lot_frontage       330
alley             1911
mas_vnr_type        22
mas_vnr_area        22
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_sf_1         1
bsmtfin_type_2      56
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
bsmt_full_bath       2
bsmt_half_bath       2
fireplace_qu      1000
garage_type        113
garage_yr_blt      114
garage_finish      114
garage_cars          1
garage_area          1
garage_qual        114
garage_cond        114
pool_qc           2042
fence             1651
misc_feature      1986
dtype: int64

In [9]:
cols = ['mas_vnr_type', 'mas_vnr_area', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 
        'fireplace_qu', 'garage_type', 'garage_qual', 'garage_cond', 'garage_finish', 
        'garage_yr_blt', 'garage_cars', 'garage_area', 'pool_qc', 'alley', 'fence', 'misc_feature'] 

for col in cols:
    train[col].fillna(0, inplace=True)

- Since the null amounts for garages are the same I decided to say they were no garage and lump the null cars and area group in as well since they were missing just 1 value each 


In [10]:
train.isnull().sum()[train.isnull().sum() != 0]

lot_frontage      330
bsmtfin_type_1     55
bsmtfin_sf_1        1
bsmtfin_type_2     56
bsmtfin_sf_2        1
bsmt_unf_sf         1
total_bsmt_sf       1
bsmt_full_bath      2
bsmt_half_bath      2
dtype: int64

In [11]:
#creating a function to switch our categorical data to numerical data for quality/condition
def qual_to_num(string):
    if string == 'Ex': #represents excellent
        return 5
    elif string == 'Gd': #represents good
        return 4
    elif string == 'TA': #represents typical/average
        return 3
    elif string == 'Fa': #represents fair
        return 2
    elif string == 'Po': #represents poor
        return 1
    else:
        return 0

In [12]:
#function to change bsmt finish since we have two columns for that
def num_finish(string):
    if string == 'GLQ': #represents good living quarters
        return 6
    elif string == 'ALQ': #represents average living quarters
        return 5
    elif string == 'BLQ': #represents below average living quarters
        return 4
    elif string == 'Rec': #represents average rec room
        return 3
    elif string == 'LwQ': #represents low quality
        return 2
    elif string == 'Unf': #represents unfinished
        return 1
    else: #0's for no basement
        return 0

In [13]:
#function for changing electrical categories to numerical values
def elec_type(string):
    if string == 'SBrKr': #represents standard circuit breakers & Romex
        return 5
    elif string == 'FuseA': #represents average fusebox
        return 4
    elif string == 'FuseF': #represents fair fusebox
        return 3
    elif string == 'FuseP': #represents poor fusebox
        return 2
    elif string == 'Mix': #represents mixed
        return 1
    else:
        return 0
        

In [14]:
#function for numerically ranking house functionality
def house_func(string):
    if string == 'Typ': #represents typical functionality
        return 7
    elif string == 'Min2': #represents minor deductions 2
        return 6
    elif string == 'Min1': #represents minor deductions 1
        return 5
    elif string == 'Mod': #represents moderate deductions
        return 4
    elif string == 'Maj1': #represents major deductions 1
        return 3
    elif string == 'Maj2': #represents major deductions 2
        return 2
    elif string == 'Sev': #represents severley damaged
        return 1
    else: #placing 0's where value is salvaged
        return 0

In [15]:
#creating function for utilities values
def utilities_type(string):
    if string == 'AllPub': #represents all public utilities
        return 4
    elif string == 'NoSewr': #represents electrical, water, gas only
        return 3
    elif string == 'NoSeWa': #represents electrical and gas only
        return 2
    elif string == 'ELO': #represents electrical only
        return 1
    else:
        return 0

In [16]:
#function for basement exposure to rate numerically
def base_expose(string):
    if string == 'Gd': #represents good exposure
        return 4
    elif string == 'Av': #represents average exposure
        return 3
    elif string == 'Mn': #represents minimum exposure
        return 2
    elif string == 'No': #represents no exposure
        return 1
    else: #0 for no basement
        return 0

In [17]:
#function for garage finish to rate numerically
def garage_fin_num(string):
    if string == 'Fin': #represents finished
        return 3
    elif string == 'RFn': #represents rough finished
        return 2
    elif string == 'Unf': #represents unfinished
        return 1
    else: #0 for no garage
        return 0

In [18]:
#function for paved driveway to represent numerically
def paved_drive(string):
    if string == 'Y': #represents paved
        return 2
    elif string == 'P': #represents partially paved
        return 1
    else: #0 for gravel/no pavement
        return 0

In [19]:
#function to transform ordinal lot shape to rank numerically
def lot_shape(string):
    if string == 'Reg': #represents regular
        return 4
    elif string == 'IR1': #represents slightly irregular
        return 3
    elif string == 'IR2': #represents moderately irregular
        return 2
    elif string == 'IR3': #represents irregular
        return 1
    else:
        return 0

In [20]:
#function to transform ordinal slope type to numerical
def slope_type(string):
    if string == 'Gtl': #represents gentle slope
        return 3
    elif string == 'Mod': #represents moderate slope
        return 2
    elif string == 'Sev': #represents severe slope
        return 1
    else:
        return 0

In [21]:
#function to transform ordinal fence ranking to numerical
def fence_num(string):
    if string == 'GdPrv': #represents good privacy
        return 4
    elif string == 'MnPrv': #represents minimum privacy
        return 3
    elif string == 'GdWo': #represents good wood
        return 2
    elif string == 'MnWw': #represents minimum wood
        return 1
    else:
        return 0

### Converting ordinal data with the fucntions from above 

In [22]:
#iterating through each quality/condition column in order to call the function to switch each value to it's corresponding number
for col in ['exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond', 'kitchen_qual', 
            'fireplace_qu', 'garage_qual', 'garage_cond', 'heating_qc', 'pool_qc']:
    train[col] = train[col].map(qual_to_num)

In [23]:
train['exter_cond'].value_counts() #checking to see if function worked

3    1778
4     215
2      49
5       7
1       2
Name: exter_cond, dtype: int64

In [24]:
#iterating through each basement finish type to call function to switch values
for col in ['bsmtfin_type_1', 'bsmtfin_type_2']:
    train[col] = train[col].map(num_finish)

In [25]:
train['bsmtfin_type_1'].value_counts() #checking to see if function worked

6    615
1    603
5    293
4    200
3    183
2    102
0     55
Name: bsmtfin_type_1, dtype: int64

In [26]:
#calling function to switch electrical values to the new numerical values
train['electrical'] = train['electrical'].map(elec_type) 

In [27]:
train['electrical'].value_counts() #checking to see if function worked

0    1868
4     140
3      35
2       7
1       1
Name: electrical, dtype: int64

In [28]:
#now calling the function for house functionality to change ordinal category values to ordinal numeric values
train['functional'] = train['functional'].map(house_func)

In [29]:
train['functional'].value_counts() #making sure function worked

7    1915
5      42
6      42
4      29
3      12
2       7
1       2
0       2
Name: functional, dtype: int64

In [30]:
train['utilities'] = train['utilities'].map(utilities_type)

In [31]:
train['utilities'].value_counts()

4    2049
3       1
2       1
Name: utilities, dtype: int64

In [32]:
train['bsmt_exposure'] = train['bsmt_exposure'].map(base_expose)

In [33]:
train['bsmt_exposure'].value_counts()

1    1339
3     288
4     203
2     163
0      58
Name: bsmt_exposure, dtype: int64

In [34]:
train['garage_finish'] = train['garage_finish'].map(garage_fin_num)

In [35]:
train['garage_finish'].value_counts()

1    849
2    579
3    509
0    114
Name: garage_finish, dtype: int64

In [36]:
train['paved_drive'] = train['paved_drive'].map(paved_drive)

In [37]:
train['paved_drive'].value_counts()

2    1861
0     151
1      39
Name: paved_drive, dtype: int64

In [38]:
train['lot_shape'] = train['lot_shape'].map(lot_shape)

In [39]:
train['lot_shape'].value_counts()

4    1295
3     692
2      55
1       9
Name: lot_shape, dtype: int64

In [40]:
train['land_slope'] = train['land_slope'].map(slope_type)

In [41]:
train['land_slope'].value_counts()

3    1953
2      88
1      10
Name: land_slope, dtype: int64

In [42]:
train['fence'] = train['fence'].map(fence_num)

In [43]:
train['fence'].value_counts()

0    1651
3     227
4      83
2      80
1      10
Name: fence, dtype: int64

### Addressing other null values

In [44]:
train.isnull().sum()[train.isnull().sum()!=0]

lot_frontage      330
bsmtfin_sf_1        1
bsmtfin_sf_2        1
bsmt_unf_sf         1
total_bsmt_sf       1
bsmt_full_bath      2
bsmt_half_bath      2
dtype: int64

In [45]:
train.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,0,3,Lvl,...,0,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,0,3,Lvl,...,0,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,0,4,Lvl,...,0,0,0,0,0,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,0,4,Lvl,...,0,0,0,0,0,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,0,3,Lvl,...,0,0,0,0,0,0,3,2010,WD,138500


In [46]:
train['lot_frontage'].fillna(train['lot_frontage'].mean(), inplace=True)

Decided to impute lont frontage with the average occuring value since most lots belong to the same residential type. I felt like that might be a safe impute. I don't think I'll end up using Lot frontage in my model but wanted to take care of nulls just incase I decide to.

In [47]:
train.isnull().sum()[train.isnull().sum()!=0]

bsmtfin_sf_1      1
bsmtfin_sf_2      1
bsmt_unf_sf       1
total_bsmt_sf     1
bsmt_full_bath    2
bsmt_half_bath    2
dtype: int64

In [48]:
train.fillna(0, inplace=True)

Deciding to impute these last few null values with zeros since there is so little it should not impact our results.

In [85]:
train.isnull().sum()[train.isnull().sum()!=0] #last check to make sure nulls are taken care of

Series([], dtype: int64)

#### Some final cleaning up of columns

In [50]:
#similar to a code we did in class, I'm going to create a loop for all columns with bathroom in it to get a total bathroom count
[col for col in train.columns if 'bath' in col]


['bsmt_full_bath', 'bsmt_half_bath', 'full_bath', 'half_bath']

In [51]:
#creating a column that adds up total amount of bathrooms
train['total_baths'] = train['bsmt_full_bath'] + (train['bsmt_half_bath'] * 0.5) + train['full_bath'] + (train['half_bath'] * 0.5)

In [54]:
train['central_air'] = pd.get_dummies(train['central_air']) #dummying up central air since it is either yes or no

In [55]:
train['street'] = pd.get_dummies(train['street'])

In [56]:
train['alley'] = train['alley'].map({'Grvl':1, 'Pave':1, 0:0})

In [87]:
train.head() #one final check to make sure data is cleaned the way I wanted

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice,total_baths
0,109,533352170,60,RL,69.0552,13517,0,0,3,Lvl,...,0,0,0,0,0,3,2010,WD,130500,2.5
1,544,531379050,60,RL,43.0000,11492,0,0,3,Lvl,...,0,0,0,0,0,4,2009,WD,220000,3.5
2,153,535304180,20,RL,68.0000,7922,0,0,4,Lvl,...,0,0,0,0,0,1,2010,WD,109000,2.0
3,318,916386060,60,RL,73.0000,9802,0,0,4,Lvl,...,0,0,0,0,0,4,2010,WD,174000,2.5
4,255,906425045,50,RL,82.0000,14235,0,0,3,Lvl,...,0,0,0,0,0,3,2010,WD,138500,2.0


In [57]:
train.to_csv('./datasets/train_clean.csv')

## Cleaning testing data

- In this section I will be taking the same steps that I did for the training data.

In [58]:
test = pd.read_csv('./datasets/test.csv')

In [59]:
test.shape

(878, 80)

In [60]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [61]:
test.columns = test.columns.str.replace(' ', '_')
test.columns = test.columns.str.lower()

### Test data set null values

In [62]:
test.isnull().sum()[test.isnull().sum()!=0]

lot_frontage      160
alley             820
mas_vnr_type        1
mas_vnr_area        1
bsmt_qual          25
bsmt_cond          25
bsmt_exposure      25
bsmtfin_type_1     25
bsmtfin_type_2     25
electrical          1
fireplace_qu      422
garage_type        44
garage_yr_blt      45
garage_finish      45
garage_qual        45
garage_cond        45
pool_qc           874
fence             706
misc_feature      837
dtype: int64

In [63]:
cols = ['mas_vnr_type', 'mas_vnr_area', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 
        'fireplace_qu', 'garage_type', 'garage_qual', 'garage_cond', 'garage_finish', 
        'garage_yr_blt', 'pool_qc', 'alley', 'fence', 'misc_feature'] 

for col in cols:
    test[col].fillna(0, inplace=True)

In [64]:
test.isnull().sum()[test.isnull().sum()!=0]

lot_frontage      160
bsmtfin_type_1     25
bsmtfin_type_2     25
electrical          1
dtype: int64

In [65]:
test['lot_frontage'].fillna(test['lot_frontage'].mean(), inplace=True)

In [66]:
for col in ['bsmtfin_type_1', 'bsmtfin_type_2']:
    test[col] = test[col].map(num_finish)

In [67]:
test['electrical'] = test['electrical'].map(elec_type)

In [68]:
test.isnull().sum()[test.isnull().sum() !=0]

Series([], dtype: int64)

### Converting ordinal data to numerical

- Using the same functions that I used in the training data cleanup

In [69]:
for col in ['exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond', 'kitchen_qual', 
            'fireplace_qu', 'garage_qual', 'garage_cond', 'heating_qc', 'pool_qc']:
    test[col] = test[col].map(qual_to_num)

In [88]:
test['exter_qual'].value_counts() #since I know the functions work, I will be just checking once in this section

3    552
4    292
5     25
2      9
Name: exter_qual, dtype: int64

In [70]:
test['functional'] = test['functional'].map(house_func)

In [71]:
test['utilities'] = test['utilities'].map(utilities_type)

In [72]:
test['bsmt_exposure'] = test['bsmt_exposure'].map(base_expose)

In [73]:
test['garage_finish'] = test['garage_finish'].map(garage_fin_num)

In [74]:
test['paved_drive'] = test['paved_drive'].map(paved_drive)

In [75]:
test['central_air'] = pd.get_dummies(test['central_air'])

In [76]:
test['street'] = pd.get_dummies(test['street'])

In [77]:
test['lot_shape'] = test['lot_shape'].map(lot_shape)

In [78]:
test['land_slope'] = test['land_slope'].map(slope_type)

In [79]:
test['alley'] = test['alley'].map({'Grvl':1, 'Pave':1, 0:0})

In [80]:
test['fence'] = test['fence'].map(fence_num)

In [81]:
test['total_baths'] = test['bsmt_full_bath'] + (test['bsmt_half_bath'] * 0.5) + test['full_bath'] + (test['half_bath'] * 0.5)


In [82]:
test.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,total_baths
0,2658,902301120,190,RM,69.000000,9142,0,1,4,Lvl,...,0,0,0,0,0,0,4,2006,WD,2.0
1,2718,905108090,90,RL,69.545961,9662,0,0,3,Lvl,...,0,0,0,0,0,0,8,2006,WD,2.0
2,2414,528218130,60,RL,58.000000,17104,0,0,3,Lvl,...,0,0,0,0,0,0,9,2006,New,3.5
3,1989,902207150,30,RM,60.000000,8520,0,0,4,Lvl,...,0,0,0,0,0,0,7,2007,WD,1.0
4,625,535105100,20,RL,69.545961,9500,0,0,3,Lvl,...,185,0,0,0,0,0,7,2009,WD,2.5


In [83]:
test.to_csv('./datasets/test_clean.csv')